In [ ]:
import numpy as np
import time
import pickle
from scipy import misc
import matplotlib
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
font = font_manager.FontProperties(style='normal', size=20)
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
%matplotlib inline
# matplotlib.use('Qt5Agg')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
import pandas as pd
#import cvxpy as cp
from scipy.optimize import fsolve
from IPython.display import Markdown
torch.set_default_dtype(torch.float64)


# Price impact in the limit order book 

$\inf\left\{\mathbb{E}\left[\sum_{n=0}^{N} \left(D_n+\frac{\kappa}{2}x_n\right)x_n + \right]:\sum_{n=0}^N x_n = X_0\right\}$

# Parameters


In [ ]:
lob_params = {
    'T':1,#time horizon
    'mu':.3,
    'gamma':.0,#permanent price impact zero
    'kappa':1.,#temporary price impact
    'rho':5.,#resilience rate
    'sigma':.3,#volatility
    'mu':.3,#drift
    'A0':0.,# initial asset price
    'balance':1e5,# initial balance
}
approx_params = {
    'num_trajectories': 1000,#  number of simulated trajectories
    'num_epochs': 3000, # number of epochs for training
    'num_time_steps': 10, # number of time steps in each trajectory
    'num_neurons': 16,# number of neurons in the hidden layers
}

# Closed-form

In [ ]:
class solution(object):
    def __init__(self,lob_params,approx_params):
        self.num_steps = approx_params['num_time_steps']
        self.T = lob_params['T']
        self.rho = lob_params['rho']
        self.kappa = lob_params['kappa']
        self.delta = self.T/self.num_steps
        self.alpha=np.exp(-self.rho*self.delta)
    def optimal(self,x_init):
        t = torch.tensor([i*self.delta for i in range(self.num_steps+1)])
        for i in range(x_init.shape[0]):
            if i==0:
                trade = ((1-(torch.where(t==0.0*self.delta,0,1)*torch.where(t==self.num_steps*self.delta,0,1))*self.alpha)*x_init[i]/((self.num_steps-1)*(1-self.alpha)+2)).unsqueeze(0)
            else:
                trade = torch.cat((trade,((1-(torch.where(t==0.0*self.delta,0,1)*torch.where(t==self.num_steps*self.delta,0,1))*self.alpha)*x_init[i]/((self.num_steps-1)*(1-self.alpha)+2)).unsqueeze(0)),axis=0)
        return trade
    def __call__(self, x):
        exec =  self.optimal(x)   
        c_tmp = torch.tensor([0.0])
        D_tmp = torch.tensor([0.0])
        for i in range(self.num_steps+1):
            c_tmp = c_tmp + D_tmp*exec[:,i]+(self.kappa/2.0)*np.power(exec[:,i],2)
            D_tmp = (D_tmp + self.kappa * exec[:,i])*self.alpha
        return c_tmp

# Coarse PGM

## Neural net for trading

In [ ]:
class trade_net(nn.Module): #NN for trading strategy
    def __init__(self,params):
        self.dim = params['dim']
        self.num_neurons = params['num_neurons']
        super(trade_net, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(self.dim, self.num_neurons),
            torch.nn.ReLU(),
            nn.Linear(self.num_neurons, self.num_neurons),
            torch.nn.ReLU(),
            nn.Linear(self.num_neurons,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])      

## Neural net for value function 

In [ ]:
class value_fnc(nn.Module): #NN for trading strategy
    def __init__(self,params):
        self.dim = params['dim']
        self.num_neurons = params['num_neurons']
        super(value_fnc, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(self.dim, self.num_neurons),
            torch.nn.ReLU(),
            nn.Linear(self.num_neurons, self.num_neurons),
            torch.nn.ReLU(),
            nn.Linear(self.num_neurons,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])      

In [ ]:
class optimal_execution(object):
    def __init__(self,lob_params,approx_params):
        self.epoch = 0
        self.loss_epoch=[]
        self.M = approx_params['num_trajectories']
        self.ite = approx_params['num_time_steps']
        self.dim = approx_params['num_neurons']
        self.T = lob_params['T']
        self.sigma = lob_params['sigma']
        self.mu = lob_params['mu']
        self.A0 =lob_params['A0'] #initial fundamental price
        self.gamma = lob_params['gamma'] # permanent price impact
        self.kappa = lob_params['kappa'] #price impact coeff
        self.rho = lob_params['rho'] #resillience
        self.X0 = lob_params['balance']
        self.neuron_model_psi = approx_params['num_neurons']
        self.delta = self.T/self.ite
        self.alpha = np.exp(-self.rho*self.delta)
        self.trade_size = trade_net(approx_params)
        self.V = value_fnc(approx_params)
        t=torch.zeros([self.M,1])
        D=torch.zeros([self.M,1])
        R=self.X0*0.9+(self.X0*1.1-self.X0*0.9)*torch.rand(self.M,1)  #remaining balance R_t   #To get a positive solution R_t has to be greater than D_t
        self.x=torch.cat((t,D,R),dim=1)
        self.trained = False
        # self.exit_dict = params # Write the result in this dict
        self.cf = solution(lob_params, approx_params)#closed-form solution
        
        
    # undate state process for one step
    def update(self,x,psi):
        t=(x[:,0]+self.delta)
        D = (x[:,1]+self.kappa*psi)*self.alpha
        R= x[:,2]-psi
        up=torch.cat((t.unsqueeze(1),D.unsqueeze(1),R.unsqueeze(1)),dim=1)
        return up
    # running cost function for one step
    def step_cost(self,x,psi):
        loss=(x[:,1]*psi+(self.kappa/2.0)*torch.pow(psi,2))
        return loss
    # summarize the trade size, cost and update for one step
    def unit(self,x):
        psi=self.trade_size(x).squeeze(1)
        loss=self.step_cost(x,psi)
        upd=self.update(x,psi)
        return psi, loss, upd
    # total cost
    def cost(self,x):
        cost=torch.zeros(self.M,self.ite+1)
        psi=torch.zeros(self.M,self.ite)
        u=x
        for i in range(self.ite+1):
            if(i!=self.ite):
                psi_run,loss_run,u_run=self.unit(u)
                #print('los func=',psi_run.shape,loss_run.shape)
                cost[:,i]=loss_run
                #print(loss)
                psi[:,i]=psi_run
                #print(psi)
                u=u_run
                #print(u)
            else:
                # print(torch.sum(psi,dim=1).shape,x[:,2].shape)
                # psi_ter=x[:,2]-torch.sum(psi,dim=1)
                # psi_ter = u[:,2]
                cost[:,i]=self.step_cost(u,u[:,2])
                #print('ter',loss_ter.shape)
        cost=torch.sum(cost,dim=1)
        #print(loss.shape)
        return cost
    
    def train(self, lr, err, num_epochs):
        start=time.time()
        # lr = 8e-3
        epoch=self.epoch
        # num_epochs=500
        L_=-1000
        optimizer = optim.Adam(self.trade_size.parameters(), lr)
        L=100000
        while (np.abs(L_-L)/np.abs(L_)>err) &  (epoch <= num_epochs):
            t0 = time.time()
            optimizer.zero_grad()
            cost=self.cost(self.x)
            loss = torch.mean(cost)
            loss.backward()
            optimizer.step()
            L = loss.clone().detach().numpy()
            self.loss_epoch.append(L)
            if epoch>0:
                L_ = self.loss_epoch[epoch-1]
            if (epoch % int(num_epochs/5)== int(num_epochs/5)-1) | (epoch == 0):
                print("At epoch {:,} the mean cost is {:.10E}.  Epoch training time = {:.2E} ms".format(epoch+1,loss.detach(),1000*(time.time()-t0)))            
            epoch=epoch+1
        end=time.time()
        print('time elapsed = {:.2e} ms'.format((end-start)*1000))
        print("Relative change in loss = %{:.7E} , last epoch = {}.".format((100*np.abs(L_-L)/np.abs(L_)),epoch+1))        
        
        
    def gen_data(self):
        if self.trained:
            cost=torch.zeros(self.M,self.ite+1)
            psi=torch.zeros(self.M,self.ite)
            u=self.x
            self.x_data = u
            for i in range(self.ite+1):
                if(i!=self.ite):
                    if i >0:
                        self.x_data = torch.cat((self.x_data,u),axis=0)
                    psi_run,loss_run,u_run=self.unit(u) 
                    #print('los func=',psi_run.shape,loss_run.shape)
                    cost[:,i]=loss_run
                    #print(loss)
                    psi[:,i]=psi_run
                    #print(psi)
                    u=u_run
                    #print(u)
                else:
                    # print(torch.sum(psi,dim=1).shape,x[:,2].shape)
                    # psi_ter=self.x[:,2]-torch.sum(psi,dim=1)
                    cost[:,i]=self.step_cost(u,u[:,2])
                    self.x_data = torch.cat((self.x_data,u),axis=0)
                    #print('ter',loss_ter.shape)
            self.y_data=torch.sum(cost,dim=1).unsqueeze(-1)
            for i in range(self.ite):
                self.y_data = torch.cat((self.y_data,torch.sum(cost[:,i+1:],dim=1).unsqueeze(-1)),axis=0)
            r = torch.randperm(self.x_data.shape[0])
            self.x_data = self.x_data[r,:].clone().detach()
            self.y_data = self.y_data[r,:].clone().detach()  
            self.exit_dict['data'] = self.x_data     
        else:
            self.train(8e-3,1e-7,3000)
            self.gen_data()        
